In [28]:
import pandas as pd
import configparser
import os

# Read the value from the config.ini file
config = configparser.ConfigParser()
try:
    config.read('config.ini')
    print(config.get('data', 'file_path1'))
except:
    print('Error reading the config file')

try:
    file_path = config.get('data', 'file_path1')
    if not os.path.isfile(file_path):
        raise FileNotFoundError('File not found')
    # Read the data from the csv file
    df = pd.read_csv(file_path)

except FileNotFoundError as e:
    file_path = config.get('data', 'file_path2')
    if not os.path.isfile(file_path):
        raise FileNotFoundError('File not found')
    # Read the data from the csv file
    df = pd.read_csv(file_path)
except:
    print('Error reading the file path from the config file')

if df is not None:
    print(df.head())
else:
    print('DataFrame is not defined')

C:\\DataExport\\VN30F1M.csv
    Symbol    Date   Open   High    Low  Close  Volume           RSI  \
0  VN30F1M  170810  757.9  757.9  745.5  745.9     367 -1.000000e+10   
1  VN30F1M  170811  746.0  746.4  743.4  745.7     510 -1.000000e+10   
2  VN30F1M  170814  745.9  748.9  745.7  748.9     707 -1.000000e+10   
3  VN30F1M  170815  750.3  750.9  748.0  748.0     946 -1.000000e+10   
4  VN30F1M  170816  747.7  748.4  745.1  748.1    1065 -1.000000e+10   

           MACD        Signal          MA10          MA20         MA100  \
0 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10   
1 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10   
2 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10   
3 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10   
4 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10 -1.000000e+10   

          MA200           MFI      Ultimate  
0 -1.000000e+10 -1.000000e+10 -1.000000e+1

In [29]:
# Drop the rows with missing values
df.dropna(inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716 entries, 0 to 1715
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Symbol    1716 non-null   object 
 1   Date      1716 non-null   int64  
 2   Open      1716 non-null   float64
 3   High      1716 non-null   float64
 4   Low       1716 non-null   float64
 5   Close     1716 non-null   float64
 6   Volume    1716 non-null   int64  
 7   RSI       1716 non-null   float64
 8   MACD      1716 non-null   float64
 9   Signal    1716 non-null   float64
 10  MA10      1716 non-null   float64
 11  MA20      1716 non-null   float64
 12  MA100     1716 non-null   float64
 13  MA200     1716 non-null   float64
 14  MFI       1716 non-null   float64
 15  Ultimate  1716 non-null   float64
dtypes: float64(13), int64(2), object(1)
memory usage: 214.6+ KB
None


In [30]:
import pandas as pd
import numpy as np

# Assuming your dataframe is named "df"
# Convert the "Date" column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Map the day of the week to corresponding labels
day_labels = ['Mo', 'Tu', 'We', 'Th', 'Fr', 'Sa', 'Su']
df['DoW'] = df['Date'].dt.dayofweek.map(lambda x: day_labels[x])

# Check if the date is the 3rd Thursday of the month and label it as "Th-expired"
DoWExp = config.get('derivatives', 'DoWExp')

df['DoW'] = np.where((df['Date'].dt.weekday == 3) & (df['Date'].dt.day <= 21) & (df['Date'].dt.day >= 15), DoWExp + '-exp', df['DoW'])

# Revert the "Date" column back to the original format
df['Date'] = df['Date'].dt.strftime('%y%m%d')

# Print the updated dataframe
df.head()


,Symbol,Date,Open,High,Low,Close,Volume,RSI,MACD,Signal,MA10,MA20,MA100,MA200,MFI,Ultimate,DoW
0,VN30F1M,170810,757.9,757.9,745.5,745.9,367,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Th
1,VN30F1M,170811,746.0,746.4,743.4,745.7,510,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Fr
2,VN30F1M,170814,745.9,748.9,745.7,748.9,707,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Mo
3,VN30F1M,170815,750.3,750.9,748.0,748.0,946,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Tu
4,VN30F1M,170816,747.7,748.4,745.1,748.1,1065,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,We


In [31]:
import pandas as pd

# Method 1: Buying at the close and selling at the open the next day
df_method1 = df.copy()
df_method1['Entry'] = df_method1['Close'].shift()
df_method1['Exit'] = df_method1['Open']
df_method1['Points'] = df_method1['Exit'] - df_method1['Entry']
cumulative_points_method1 = df_method1['Points'].cumsum().iloc[-1]

# Method 2: Shorting at the open plus 2 points and covering at the close
df_method2 = df.copy()
df_method2['Entry'] = df_method2['Open'] + 2
df_method2['Exit'] = df_method2['Close']
df_method2['Points'] = df_method2['Exit'] - df_method2['Entry']
cumulative_points_method2 = df_method2['Points'].cumsum().iloc[-1]

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df.copy()
df_method3['Entry'] = df_method3['Open'] - 2
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']
cumulative_points_method3 = df_method3['Points'].cumsum().iloc[-1]

# Method 4: Buying at open and selling at close if previous day have Close > MA(close, 10). Shorting at open and covering at close if previous day have Close < MA(close, 10)
df_method4 = df.copy()
df_method4['MA10'] = df_method4['Close'].rolling(window=10).mean()
df_method4['Entry'] = df_method4['Open']
df_method4['Exit'] = df_method4['Close']
df_method4['Points'] = np.where(df_method4['Close'].shift() > df_method4['MA10'].shift(), df_method4['Exit'] - df_method4['Entry'], df_method4['Entry'] - df_method4['Exit'])
cumulative_points_method4 = df_method4['Points'].cumsum().iloc[-1]

# Method 5: Buying at the previous close minus 2 points and selling at the close
df_method5 = df.copy()
df_method5['Entry'] = df_method5['Close'].shift() - 2
df_method5['Exit'] = df_method5['Close']
df_method5['Points'] = df_method5['Exit'] - df_method5['Entry']
cumulative_points_method5 = df_method5['Points'].cumsum().iloc[-1]

# Compare the cumulative points for each method
method_points = {
    'Method 1': cumulative_points_method1,
    'Method 2': cumulative_points_method2,
    'Method 3': cumulative_points_method3,
    'Method 4': cumulative_points_method4,
    'Method 5': cumulative_points_method5
}

# Find the method with the most points
best_method = max(method_points, key=method_points.get)
best_points = method_points[best_method]

# Print the results
for method, points in method_points.items():
    print(f"{method}: Cumulative Points = {points}")

print(f"\nBest Method: {best_method}")
print(f"Cumulative Points: {best_points}")


Method 1: Cumulative Points = 769.0
Method 2: Cumulative Points = -3638.0000000000027
Method 3: Cumulative Points = 3225.9999999999964
Method 4: Cumulative Points = 506.79999999999757
Method 5: Cumulative Points = 4004.999999999998

Best Method: Method 5
Cumulative Points: 4004.999999999998


In [32]:
# Optimize the moving average window length in Method 4
cumulative_points = {}
for window in range(5, 21):
    df_method4 = df.copy()
    df_method4['MA'] = df_method4['Close'].rolling(window=window).mean()
    df_method4['Entry'] = df_method4['Open']
    df_method4['Exit'] = df_method4['Close']
    df_method4['Points'] = np.where(df_method4['Close'].shift() > df_method4['MA'].shift(), df_method4['Exit'] - df_method4['Entry'], df_method4['Entry'] - df_method4['Exit'])
    cumulative_points[window] = df_method4['Points'].cumsum().iloc[-1]

# Find the window length with the most points
best_window = max(cumulative_points, key=cumulative_points.get)
best_points = cumulative_points[best_window]

# Print the results
for window, points in cumulative_points.items():
    print(f"Window Length: {window}, Cumulative Points = {points}")


Window Length: 5, Cumulative Points = 624.1999999999997
Window Length: 6, Cumulative Points = 868.7999999999989
Window Length: 7, Cumulative Points = 894.3999999999983
Window Length: 8, Cumulative Points = 998.3999999999985
Window Length: 9, Cumulative Points = 637.7999999999973
Window Length: 10, Cumulative Points = 506.79999999999757
Window Length: 11, Cumulative Points = 603.7999999999982
Window Length: 12, Cumulative Points = 454.3999999999977
Window Length: 13, Cumulative Points = 385.7999999999978
Window Length: 14, Cumulative Points = 287.39999999999725
Window Length: 15, Cumulative Points = 298.39999999999657
Window Length: 16, Cumulative Points = 271.199999999997
Window Length: 17, Cumulative Points = 363.99999999999625
Window Length: 18, Cumulative Points = 381.1999999999963
Window Length: 19, Cumulative Points = 445.7999999999955
Window Length: 20, Cumulative Points = 505.79999999999484


In [33]:
import pandas as pd

# Convert the Date column to string format
df['Date'] = df['Date'].astype(str)

# Filter the dataframe for the desired date range (2022 and 2023)
df_filtered = df[(df['Date'] >= '220101') & (df['Date'] <= '240601')]

# Method 1: Buying at the close and selling at the open the next day
df_method1 = df_filtered.copy()
df_method1['Entry'] = df_method1['Close'].shift()
df_method1['Exit'] = df_method1['Open']
df_method1['Points'] = df_method1['Exit'] - df_method1['Entry']

# Calculate cumulative points for Method 1
if len(df_method1) > 0:
    cumulative_points_method1 = df_method1['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method1 = 0

# Method 2: Shorting at the open plus 2 points and covering at the close
df_method2 = df_filtered.copy()
df_method2['Entry'] = df_method2['Open'] + 2
df_method2['Exit'] = df_method2['Close']
df_method2['Points'] = df_method2['Exit'] - df_method2['Entry']

# Calculate cumulative points for Method 2
if len(df_method2) > 0:
    cumulative_points_method2 = df_method2['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method2 = 0

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df_filtered.copy()
df_method3['Entry'] = df_method3['Open'] - 2
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']

# Calculate cumulative points for Method 3
if len(df_method3) > 0:
    cumulative_points_method3 = df_method3['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method3 = 0

# Method 4: Buying at open and selling at close if previous day have Close > MA(close, 5). Shorting at open and covering at close if previous day have Close < MA(close, 5)
df_method4 = df_filtered.copy()
df_method4['MA5'] = df_method4['Close'].rolling(window=5).mean()
df_method4['Entry'] = df_method4['Open']
df_method4['Exit'] = df_method4['Close']
df_method4['Points'] = np.where(df_method4['Close'].shift() > df_method4['MA5'].shift(), df_method4['Exit'] - df_method4['Entry'], df_method4['Entry'] - df_method4['Exit'])

# Calculate cumulative points for Method 4
if len(df_method4) > 0:
    cumulative_points_method4 = df_method4['Points'].cumsum().iloc[-1]
else:
    cumulative_points_method4 = 0

# Method 5: Buying at the previous close minus 2 points and selling at the close
df_method5 = df_filtered.copy()
df_method5['Entry'] = df_method5['Close'].shift() - 2
df_method5['Exit'] = df_method5['Close']
df_method5['Points'] = df_method5['Exit'] - df_method5['Entry']

# Calculate cumulative points for Method 5
if len(df_method5) > 0:
    cumulative_points_method5 = df_method5['Points'].cumsum().iloc[-1]

# Compare the cumulative points for each method
method_points = {
    'Method 1': cumulative_points_method1,
    'Method 2': cumulative_points_method2,
    'Method 3': cumulative_points_method3,
    'Method 4': cumulative_points_method4,
    'Method 5': cumulative_points_method5
}

# Find the method with the most points
best_method = max(method_points, key=method_points.get)
best_points = method_points[best_method]

# Print the results
for method, points in method_points.items():
    print(f"{method}: Cumulative Points = {points}")

print(f"\nBest Method: {best_method}")
print(f"Cumulative Points: {best_points}")


Method 1: Cumulative Points = 189.70000000000277
Method 2: Cumulative Points = -1660.6000000000026
Method 3: Cumulative Points = 731.3999999999971
Method 4: Cumulative Points = 239.79999999999973
Method 5: Cumulative Points = 905.5

Best Method: Method 5
Cumulative Points: 905.5


In [34]:
### METHOD 3
import pandas as pd

# Convert the Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')

# Filter the dataframe for the desired date range (2022 and 2023 and 2024)
df_filtered = df[(df['Date'].dt.year.isin([2022, 2023, 2024]))]

# Method 3: Buying at the open minus 2 points and selling at the close
df_method3 = df_filtered.copy()
df_method3['Entry'] = df_method3['Open'] - 3
df_method3['Exit'] = df_method3['Close']
df_method3['Points'] = df_method3['Exit'] - df_method3['Entry']

# Calculate cumulative points for Method 3
cumulative_points_method3 = df_method3['Points'].cumsum()

# Group the data by month and year and calculate the points earned for each month and year
df_method3['Month'] = df_method3['Date'].dt.month
df_method3['Year'] = df_method3['Date'].dt.year
monthly_points = df_method3.groupby(['Year', 'Month'])['Points'].sum()

# Print the points earned each month and year
#print("Points Earned Each Month and Year:")
#print(monthly_points)


# Show results in year 2024 only
monthly_points_2024 = monthly_points[2024]
print("Points Earned Each Month in 2024:")
print(monthly_points_2024)

Points Earned Each Month in 2024:
Month
1     84.8
2    117.9
3     76.2
4     19.3
5     82.3
6     58.9
Name: Points, dtype: float64


In [35]:
# Create a new column 'Prev_Close' which is the 'Close' column shifted by 1
df['Prev_Close'] = df['Close'].shift(1)

# Calculate the difference between 'Low' and 'Prev_Close'
df['Diff'] = df['Low'] - df['Prev_Close']

# Filter data for the year 2023
df_2023 = df[df['Date'].dt.year == 2023]

# Calculate the rate at which 'Low' is less than 'Prev_Close' by 2 points
rate = (df_2023['Diff'] <= -2).mean()

print(f"The rate at which 'Low' is less than the previous day's 'Close' by 2 points in the trading days of 2023 is {rate}")

The rate at which 'Low' is less than the previous day's 'Close' by 2 points in the trading days of 2023 is 0.714859437751004


In [36]:
# Create a new column 'Prev_Close' which is the 'Close' column shifted by 1
df['Prev_Close'] = df['Close'].shift(1)

# Calculate the difference between 'Low' and 'Prev_Close'
df['Diff'] = df['Low'] - df['Prev_Close']

# Filter data for the year 2023
df_2023 = df[df['Date'].dt.year == 2023]

# Calculate the rate at which 'Low' is less than 'Prev_Close' by 2 to 10 points
for i in range(2, 11):
    rate = (df_2023['Diff'] <= -i).mean()
    print(f"The rate at which 'Low' is less than the previous day's 'Close' by {i} points in the trading days of 2023 is {rate}")

The rate at which 'Low' is less than the previous day's 'Close' by 2 points in the trading days of 2023 is 0.714859437751004
The rate at which 'Low' is less than the previous day's 'Close' by 3 points in the trading days of 2023 is 0.6345381526104418
The rate at which 'Low' is less than the previous day's 'Close' by 4 points in the trading days of 2023 is 0.5502008032128514
The rate at which 'Low' is less than the previous day's 'Close' by 5 points in the trading days of 2023 is 0.4939759036144578
The rate at which 'Low' is less than the previous day's 'Close' by 6 points in the trading days of 2023 is 0.43373493975903615
The rate at which 'Low' is less than the previous day's 'Close' by 7 points in the trading days of 2023 is 0.40160642570281124
The rate at which 'Low' is less than the previous day's 'Close' by 8 points in the trading days of 2023 is 0.3614457831325301
The rate at which 'Low' is less than the previous day's 'Close' by 9 points in the trading days of 2023 is 0.33734939

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Create a target variable 'Action' that is 1 if the price increased compared to the previous day, and 0 otherwise
df['Action'] = (df['Close'].shift(-1) > df['Close']).astype(int)

# Use the 'Open', 'High', 'Low', 'Close' columns as features
features = df[['Open', 'High', 'Low', 'Close']]

# Use the 'Action' column as the target
target = df['Action']

# Split the data into a training set and a test set
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

# Create and train a logistic regression model
model = LogisticRegression()
model.fit(features_train, target_train)

# Make a prediction for the next day
next_day_features = features.iloc[-1].values.reshape(1, -1)
next_day_features = scaler.transform(next_day_features)
next_day_prediction = model.predict(next_day_features)

if next_day_prediction[0] == 1:
    print("The model predicts a price increase. Consider buying.")
else:
    print("The model predicts a price decrease. Consider shorting.")

The model predicts a price increase. Consider buying.


c:\Users\PC\anaconda3\envs\pyenv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [38]:
df

,Symbol,Date,Open,High,Low,Close,Volume,RSI,MACD,Signal,MA10,MA20,MA100,MA200,MFI,Ultimate,DoW,Prev_Close,Diff,Action
0,VN30F1M,2017-08-10,757.9,757.9,745.5,745.9,367,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Th,NaN,NaN,0
1,VN30F1M,2017-08-11,746.0,746.4,743.4,745.7,510,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Fr,745.9,-2.5,1
2,VN30F1M,2017-08-14,745.9,748.9,745.7,748.9,707,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Mo,745.7,0.0,0
3,VN30F1M,2017-08-15,750.3,750.9,748.0,748.0,946,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,Tu,748.9,-0.9,1
4,VN30F1M,2017-08-16,747.7,748.4,745.1,748.1,1065,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,We,748.0,-2.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,VN30F1M,2024-06-17,1315.5,1316.0,1305.0,1308.0,218205,5.505580e+01,1.230280e+01,1.396510e+01,1.310330e+03,1.298150e+03,1.254460e+03,1.197520e+03,5.435640e+01,5.415390e+01,Mo,1317.0,-12.0,1
1712,VN30F1M,2024-06-18,1312.2,1322.5,1308.5,1308.5,221206,5.522990e+01,1.142820e+01,1.398140e+01,1.311680e+03,1.297970e+03,1.255890e+03,1.198090e+03,6.217500e+01,4.860070e+01,Tu,1308.0,0.5,1
1713,VN30F1M,2024-06-19,1309.9,1325.8,1298.7,1317.0,278025,5.819500e+01,1.129080e+01,1.394420e+01,1.313880e+03,1.299210e+03,1.257330e+03,1.198620e+03,7.067870e+01,5.062780e+01,We,1308.5,-9.8,1
1714,VN30F1M,2024-06-20,1317.6,1327.9,1316.3,1323.7,217494,6.042020e+01,1.158900e+01,1.400520e+01,1.316380e+03,1.300400e+03,1.258670e+03,1.199180e+03,7.089960e+01,5.301300e+01,Th-exp,1317.0,-0.7,0
